## Weaviate Quickstart - Python code

This is an example Python (Jupyter) notebook for the [Weaviate Quickstart tutorial](https://weaviate.io/developers/weaviate/quickstart).

<a target="_blank" href="https://colab.research.google.com/github/weaviate-tutorials/quickstart/blob/main/quickstart_end_to_end.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

*Last updated: 2 Feb 2023*

In [1]:
# Install Weaviate if necessary
# !pip install -Uqq weaviate-client

Note: Remember to replace the API key, parameter name and vectorizer in the cells below to correspond to your preferred inference API.

In [2]:
# Load API token - replace with script to load your token, e.g. from your environment variables, via:
# import os
# api_tkn = os.getenv('OPENAI_APIKEY')

with open('../keys/openai_key.txt', 'r') as f:
    api_tkn = f.read().strip()

# Import libraries
import weaviate
import json

# Instantiate Weaviate client
client = weaviate.Client(
    url="https://quickstart.weaviate.network",  # Replace this with your WCS or Docker endpoint (use HTTP, not HTTPS for local addresses)
    additional_headers={
        "X-OpenAI-Api-Key": api_tkn  # Or "X-Cohere-Api-Key" or "X-HuggingFace-Api-Key" 
    }
)

In [3]:
# Delete existing schema (if necessary - THIS WILL ALSO DELETE ALL OF YOUR DATA)
client.schema.delete_all()

# Fetch & inspect schema (should be empty)
schema = client.schema.get()
print(json.dumps(schema, indent=4))

{
    "classes": []
}


In [4]:
# ===== add schema ===== 
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai"  # Or "text2vec-cohere" or "text2vec-huggingface"
}

client.schema.create_class(class_obj)

In [5]:
# ===== import data ===== 
# Load data from GitHub
import requests
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Configure a batch process
with client.batch as batch:
    batch.batch_size=100
    # Batch import all Questions
    for i, d in enumerate(data):
        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        client.batch.add_data_object(properties, "Question")

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [6]:
client.data_object.get()

{'deprecations': None,
 'objects': [{'class': 'Question',
   'creationTimeUnix': 1675400308853,
   'id': '1ce93083-0b33-43c6-9041-f5238ca337c2',
   'lastUpdateTimeUnix': 1675400308853,
   'properties': {'answer': 'the diamondback rattler',
    'category': 'ANIMALS',
    'question': 'Heaviest of all poisonous snakes is this North American rattlesnake'},
   'vectorWeights': None},
  {'class': 'Question',
   'creationTimeUnix': 1675400308853,
   'id': '53b8770a-cf7b-4e34-b4a7-127a24e96ed0',
   'lastUpdateTimeUnix': 1675400308853,
   'properties': {'answer': 'Antelope',
    'category': 'ANIMALS',
    'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa'},
   'vectorWeights': None},
  {'class': 'Question',
   'creationTimeUnix': 1675400308853,
   'id': '7727c6b1-aa9f-4e96-ac56-f13eda91d00d',
   'lastUpdateTimeUnix': 1675400308853,
   'properties': {'answer': 'the atmosphere',
    'category': 'SCIENCE',
    'question': 'Changes in the tropospheric lay

In [7]:
nearText = {"concepts": ["biology"]}

result = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text(nearText)
    .with_limit(2)
    .do()
)

print(json.dumps(result, indent=4))

{
    "data": {
        "Get": {
            "Question": [
                {
                    "answer": "DNA",
                    "category": "SCIENCE",
                    "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
                },
                {
                    "answer": "species",
                    "category": "SCIENCE",
                    "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
                }
            ]
        }
    }
}
